In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
from kfsims.node import make_simple_nodes
from kfsims.common import init_trajectory

In [27]:
traj = init_trajectory()

In [35]:
n = make_simple_nodes(1)[0]
n()
n.post_rmse(traj.X)

2.5343283205263147

In [42]:
pool = make_simple_nodes(10)
[n() for n in pool]
[n.post_rmse(traj.X) for n in pool]

[4.044015924810842,
 4.318936446731504,
 4.601427078683353,
 4.986641828133016,
 4.745506606674556,
 7.844189560477856,
 4.47968580036298,
 4.096280511910233,
 4.584693671880445,
 4.723844427980279]